# Database Setup

This notebook can be used to set up the local CN database.

### Data

Use the following links to download the relevant data and store it in the ```data``` directory:

- Relations: https://github.com/commonsense/conceptnet5/wiki/Downloads
- Embeddings: https://github.com/commonsense/conceptnet-numberbatch (deprecated)
- FastText model: https://fasttext.cc/docs/en/english-vectors.html

You may also store the files in a different directory.
If you do, please be advised that you will need to adjust the ```DATA_PATH``` value in the next step.

### Environment Variables

Set the following environment variables, e.g. by creating a ```.env``` file from the ```.env.example``` file and filling in the values:

- DB_PATH
- DATA_PATH

### Requirements

Run ```pip install -r requirements-dev.txt``` in order to install the packages necessary for running this notebook.

### Run

Run all cells in this notebook in order to create a local ConceptNet database.

This may take a while.

In [12]:
import json
import os
from pathlib import Path

import fasttext
from tqdm.notebook import tqdm

import numpy as np
import sqlite3

from conceptnet_local._cn_service import _db_adapt_array, _db_convert_array, get_all_edges

from dotenv import load_dotenv
load_dotenv()

True

## Database

In [2]:
sqlite3.register_adapter(np.ndarray, _db_adapt_array)
sqlite3.register_converter("ARRAY", _db_convert_array)

In [3]:
db_connection = sqlite3.connect(os.environ["DB_PATH"], detect_types=sqlite3.PARSE_DECLTYPES)
db_cursor = db_connection.cursor()

## Relations

In [4]:
data_dir = os.environ["DATA_PATH"]

en_relations_file = Path(f"{data_dir}/en-relations-raw.csv")
if not en_relations_file.is_file():
    with open("../data/conceptnet-assertions-5.7.0.csv", "r") as origin_file:
        with open("../data/en-relations-raw.csv", "w") as output_file:
            for line in tqdm(origin_file):
                parts = line.split("\t")
                if parts[2].startswith("/c/en/") and parts[3].startswith("/c/en/"):
                    output_file.write(line)

In [6]:
def get_cn_id_without_subs(cn_id: str) -> str:
    """Remove the sub-categories from the given CN ID."""
    split_by_slash = cn_id.split("/")  # assumed shape: /c/<lang>/<concept>[/<sub-categories>]
    if len(split_by_slash) > 4:
        split_by_slash = split_by_slash[:4]
    return "/".join(split_by_slash)

In [8]:
with open(f"{data_dir}/en-relations-raw.csv", "r") as f:
    num_relations = sum(1 for _ in f)

3423004

In [11]:
db_cursor.execute("DROP TABLE IF EXISTS relations;")
db_cursor.execute("""
CREATE TABLE relations(
    id      TEXT    NOT NULL,
    start   TEXT    NOT NULL,
    end     TEXT    NOT NULL,
    rel     TEXT    NOT NULL,
    weight  REAL    NOT NULL
);
""")
db_cursor.execute("CREATE INDEX index_relations_start ON relations(start);")
db_cursor.execute("CREATE INDEX index_relations_end ON relations(end);")

with open(f"{data_dir}/en-relations-raw.csv", "r") as raw_file:
    for line in tqdm(raw_file, total=num_relations):
        try:
            a_id, rel, start, end, data = line.split("\t")
            start = get_cn_id_without_subs(cn_id=start)
            end = get_cn_id_without_subs(cn_id=end)
            weight = json.loads(data)["weight"]
            db_cursor.execute("INSERT INTO relations VALUES (?, ?, ?, ?, ?)", (a_id, start, end, rel, weight))
        except Exception as e:
            print(line)
            raise e
        
db_connection.commit()

  0%|          | 0/3423004 [00:00<?, ?it/s]

## Concepts

In [10]:
statement = db_cursor.execute(
    "SELECT DISTINCT start from relations",
)
result_start = statement.fetchall()

statement = db_cursor.execute(
    "SELECT DISTINCT end from relations",
)
result_end = statement.fetchall()

all_concepts = {c[0] for c in result_start}.union({c[0] for c in result_end})
all_concepts = list(all_concepts)
all_concepts.sort()

In [13]:
db_cursor.execute("DROP TABLE IF EXISTS concepts;")
db_cursor.execute("""
CREATE TABLE concepts (
    id      TEXT    NOT NULL,
    degree  INTEGER NOT NULL
);
""")
db_cursor.execute("CREATE INDEX index_concepts ON concepts(id);")

for concept in tqdm(all_concepts):
    edges = get_all_edges(cn_id=concept, db_cursor=db_cursor)
    degree = len(edges)
    
    db_cursor.execute("INSERT INTO concepts VALUES (?, ?)", (concept, degree))
    db_connection.commit()

  0%|          | 0/1165190 [00:00<?, ?it/s]

## Embeddings

In [14]:
fasttext_path = os.environ["FASTTEXT_PATH"]
fasttext_model = fasttext.load_model(fasttext_path)

In [15]:
lang_prefix = "/c/en/"

In [16]:
db_cursor.execute("DROP TABLE IF EXISTS embeddings;")
db_cursor.execute("""
CREATE TABLE embeddings(
    concept_id  TEXT    NOT NULL,
    embedding   ARRAY   NOT NULL
);
""")
db_cursor.execute("CREATE UNIQUE INDEX index_embeddings_id ON embeddings(concept_id)")

for concept in tqdm(all_concepts):
    concept_natural = concept.replace(lang_prefix, "").replace("_", " ")
    embedding = fasttext_model.get_sentence_vector(text=concept_natural)
    
    db_cursor.execute("INSERT INTO embeddings VALUES (?, ?)", (concept, embedding))

db_connection.commit()

  0%|          | 0/1165190 [00:00<?, ?it/s]

## Cleanup

In [17]:
db_connection.close()